<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 4*

# 📝 Assignment

---


# Transformer_and_BERT_GPT
## Code

###문제1
"""input your code""" 부분에 알맞은 코드로 알맞게 짝지어진 것을 고르시오. (A, B, C)

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.
  
    Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth_v)
        mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.
    
    Returns:
        output, attention_weights
    """
    
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    
    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math."""input your code(A)"""(dk)
    
    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  
        
    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn."""input your code(B)"""(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    
    output = tf."""input your code(C)"""(attention_weights, v)  # (..., seq_len_q, depth_v)
    
    return output, attention_weights

### IMDB 영화 데이터 리뷰 분류

In [3]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 13.3 MB/s 
     |████████████████████████████████| 3.3 MB 39.8 MB/s 
     |████████████████████████████████| 596 kB 37.2 MB/s 
     |████████████████████████████████| 895 kB 45.4 MB/s 


In [2]:
!pip install -q datasets

     |████████████████████████████████| 290 kB 10.2 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 
     |████████████████████████████████| 1.3 MB 40.6 MB/s 
     |████████████████████████████████| 125 kB 53.0 MB/s 
     |████████████████████████████████| 243 kB 40.6 MB/s 
     |████████████████████████████████| 160 kB 47.8 MB/s 
     |████████████████████████████████| 271 kB 48.3 MB/s 


사전 학습된 언어모델을 사용하기 위해 Huggingface에서 제공하는 transformers와 datasets를 사용하겠습니다.

Huggingface에서 제공하는 datasets에는 유명한 자연어처리 벤치마크 데이터셋을 비롯해 다양한 데이터를 불러올 수 있습니다.

우리는 imdb 영화 데이터 감정 분류 태스크를 수행해보겠습니다.

이 데이터는 영화에 대한 리뷰와 감정(긍정은 1, 부정은 0)으로 이루어져 있습니다.

In [4]:
from datasets import load_dataset

imdb_datasets = load_dataset('imdb')
print(imdb_datasets)

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [5]:
print(imdb_datasets['train'][0])

{'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!', 'label': 1}


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Huggingface에서 제공하는 언어모델을 사용하려면, 해당 언어모델에 맞는 토크나이저를 사용해야 합니다.

우리는 가장 기본적인 BERT를 이용할 것이고, BERT에 맞는 토크나이저를 불러오겠습니다.

In [7]:
print('text: ', imdb_datasets['train'][0]['text'])
print(tokenizer(imdb_datasets['train'][0]['text']))

text:  Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!
{'input_ids': [101, 139, 16071, 3192, 1693, 1110, 170, 11540, 3789, 119, 1135, 1868, 1120, 1103, 1269, 1159, 1112, 1199, 1168, 2648, 1164, 1278, 1297, 117, 1216, 1112, 107, 14290, 107, 1

토크나이저를 거치면, 해당 단어별 id 값과 token_type_ids, attention_mask 값을 얻을 수 있습니다.

In [8]:
def tokenize_function(exmaples):
  return tokenizer(exmaples['text'], padding='max_length', truncation=True)

tokenized_datasets = imdb_datasets.map(tokenize_function, batched=True)
print(tokenized_datasets)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 50000
    })
})


Huggingface에서 제공하는 datasets는 map이라는 메소드를 통해 tokenize function을 모든 데이터에 대해 한번에 적용 가능합니다.

우리는 감정 분류 태스크를 처리하기 위한 전체적 맥락 파악이 우선이니, 작은 데이터셋만을 활용하겠습니다.

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### 문제2
imdb 데이터셋의 첫번째 텍스트를 토크나이저로 변환한 후, BERT 모델에 통과해보겠습니다.

BERT 모델은 last_hidden_state와 pooler_output을 반환하는데, 각각의 shape을 적으세요
- [30, 100, 700] 형태로 적어주세요

In [11]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained('bert-base-cased')
bert_result = bert_model(**tokenizer(imdb_datasets['train'][0]['text'], return_tensors='pt'))

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


우리는 TFAutoModelForSequenceClassification을 사용하여 감정분류를 해보겠습니다.

Huggingface에서는 --ModelForSequenceClassificiation 형태의 모델을 따로 제공합니다.

이는 분류를 위한 모델로, BERT와 같은 언어모델의 마지막 레이어에 분류를 위한 완전연결층이 추가된 모델입니다.

자세한 내용은 Huggingface의 공식 문서를 참고하세요

In [34]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

# 문자열 형태의 text는 학습에 필요하지 않으니, 제거해줍니다.
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
125/125 [==============================] - 332s 2s/step - loss: 0.5570 - sparse_categorical_accuracy: 0.6980 - val_loss: 0.5320 - val_sparse_categorical_accuracy: 0.7300
Epoch 2/3
125/125 [==============================] - 307s 2s/step - loss: 0.4890 - sparse_categorical_accuracy: 0.7650 - val_loss: 0.6566 - val_sparse_categorical_accuracy: 0.6500
Epoch 3/3
125/125 [==============================] - 308s 2s/step - loss: 0.3759 - sparse_categorical_accuracy: 0.8620 - val_loss: 0.4679 - val_sparse_categorical_accuracy: 0.8100
